# **THIS CODE USES THE FOUNDATION OF TRANSFER LEARNING , IT KEEPS THE BASE LAYERS OF RESNET MODEL (PRETRAINED MODEL ON MILLIONS OF IMAGE) FIXED AND CREATES THE FEATURE MAP AND TRAINS THE MODEL TO MAP PRODUCT AND ITS FEATURE**

**THIS CODE IS USED TO MAKE *product_maped_to_feature* MODEL WHICH IS USED IN SEARCH_ENGINE_MODEL.IPYNB file**


In [ ]:
import keras
from keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import pickle

# Load ResNet50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False)

# Function to preprocess and get feature maps
def get_feature_map(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content)).resize((224, 224))
    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature_map = base_model.predict(x)
    return feature_map.flatten()

# Load the CSV file


# -----------------------------------------------------------------------------------------------------------
  # PASTE THE CSV FILE OF DATA ON WHICH YOU WANT TO TRAIN MODEL
csv_file_path = '/content/all category.csv'
df = pd.read_csv(csv_file_path)
# --------------------------------------------------------------------------------------------------------



# Process each image and get its feature map
feature_maps = []

for index, row in df.iterrows():
    img_url = row['imgurl']
    try:
        feature_map = get_feature_map(img_url)
        feature_maps.append(feature_map)
    except Exception as e:
        print(f"Error processing image at {img_url}: {e}")
        feature_maps.append(None)  # Append None if there's an error

# Save the feature maps and links to a file
data = {
    'links': df['linkn'].tolist(),
    'feature_maps': feature_maps
}

with open('product_maped_to_features', 'wb') as f:
    pickle.dump(data, f)

print("Feature maps have been saved to product_maped_to_features.pkl")
